In [1]:
import pandas as pd
import numpy as np

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is [here](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh), and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [4]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09


In [17]:
tree2 = pd.DataFrame(trees)
tree2 = trees[["spc_common", "health"]]
tree3 = tree2.groupby('health').count()
tree3
#tree2.columns.to_list()

,spc_common
health,
Fair,182
Good,735
Poor,54


In [ ]:
Question 1:

In [37]:
final_df = pd.DataFrame()
boro_list = ['Bronx', 'Brooklyn', 'Manhattan', 'Staten Island', 'Queens']
species = 'American beech'
for i in boro_list:
    soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
            '$select=spc_common,health' +\
            '&$where=boroname=\'' + i + '\'' + '&spc_common=\'' + species +\
            '\'&$limit=100000').replace(' ', '%20')

    soql_trees = pd.read_json(soql_url)
    soql_trees = soql_trees.groupby('health').count()['spc_common'].reset_index()
    soql_trees['Borough'] = i
    final_df = final_df.append(soql_trees)
print(final_df)

  health  spc_common        Borough
0   Fair           7          Bronx
1   Good          21          Bronx
2   Poor           3          Bronx
0   Fair          18       Brooklyn
1   Good          59       Brooklyn
2   Poor           6       Brooklyn
0   Fair           4      Manhattan
1   Good          15      Manhattan
2   Poor           3      Manhattan
0   Fair           1  Staten Island
1   Good          46  Staten Island
2   Poor           2  Staten Island
0   Fair           8         Queens
1   Good          73         Queens
2   Poor           7         Queens


Question 2:

In [42]:
borough = 'Brooklyn'
species = 'American beech'
soql_url2 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
            '$select=steward,health,count(tree_id)' +\
            '&$where=boroname=\'' + borough + '\'' + '&spc_common=\'' + species +\
            '\'&$group=steward,health').replace(' ', '%20')

df = pd.read_json(soql_url2).dropna().rename(columns = {"count_tree_id":"Trees Count"})
#print(df)   
df["type"] = df['steward'].map(lambda x : "no steward" if x == 'None' else "steward")
print(df) 

  steward health  Trees Count        type
0    1or2   Fair           10     steward
1    1or2   Good           27     steward
2    1or2   Poor            1     steward
3    3or4   Good            2     steward
4    None   Fair            8  no steward
5    None   Good           30  no steward
6    None   Poor            5  no steward
